# NLP Chatbot

In [6]:
# Import libraries

import numpy as np
import random
import json
import pickle

from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import gradient_descent_v2

import nltk
from nltk.stem import WordNetLemmatizer

## Pre-processing data

From the intents.json file, each tag in the intents has a pattern phrase where each word needs to be extracted out of the sentence through tokenization. In order to simplify the collection of words obtained from tokenization, lemmatization is used to reduce words into their dictionary form, e.g. break, breaks, broke, broken and breaking all reduce down to break.

In [7]:
# Load file
intents_file = open('intents.json').read()
intents = json.loads(intents_file)

# Tokenization

words = []
classes = []
documents = []
ignore_letters = ['!', '?', ',', '.']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word = nltk.word_tokenize(pattern) # tokenize each word
        words.extend(word)
        documents.append((word, intent['tag'])) # add documents in the corpus
        if intent['tag'] not in classes:
            classes.append(intent['tag']) # add to classes list

# Lemmatization

wnl = WordNetLemmatizer()
lemma_words = []

for word in words:
    if word not in ignore_letters and word not in lemma_words:
        lemma_words.append(wnl.lemmatize(word.lower())) # lemmatizing each word

lemma_words = sorted(lemma_words)

classes = sorted(classes)


## Training and Testing Data

Training and testing data are created by coverting the input pattern into numbers to allow for the application of mathematical modelling. This is done by creating a list of zeros with the same length as the total number of words. A one is set to the index corresponding to the word that is contianed in the pattern. The output is set to one for the corrsponding class that the pattern belongs to.

In [8]:
training = []

empty_output = np.zeros(len(classes))

for document in documents:
    bag_of_words = [] 
    word_patterns = document[0]
    word_patterns = [wnl.lemmatize(word.lower()) for word in word_patterns]
    
    for word in words:
        bag_of_words.append(1) if word in word_patterns else bag_of_words.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    
    output_row = list(empty_output)
    output_row[classes.index(document[1])] = 1
    training.append([bag_of_words, output_row])

random.shuffle(training)

training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])


<ipython-input-8-d981ff7d8f42>:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


## Training Model

The model consists of 3 layers. The first layer has 128 neurons, the second has 64 neurons while the last layer has the same number of neurons as classes. Dropout layers assist with overfitting of the model while the SDG optimizer is used for fitting.

In [9]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0],),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = gradient_descent_v2.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)


Epoch 1/200
10/10 [==============================] - 1s 3ms/step - loss: 2.1376 - accuracy: 0.1277
Epoch 2/200
10/10 [==============================] - 0s 2ms/step - loss: 1.8542 - accuracy: 0.4043
Epoch 3/200
10/10 [==============================] - 0s 2ms/step - loss: 1.7855 - accuracy: 0.4468
Epoch 4/200
10/10 [==============================] - 0s 3ms/step - loss: 1.5399 - accuracy: 0.5319
Epoch 5/200
10/10 [==============================] - 0s 2ms/step - loss: 1.4093 - accuracy: 0.5319
Epoch 6/200
10/10 [==============================] - 0s 2ms/step - loss: 1.3612 - accuracy: 0.5532
Epoch 7/200
10/10 [==============================] - 0s 3ms/step - loss: 1.2677 - accuracy: 0.5745
Epoch 8/200
10/10 [==============================] - 0s 2ms/step - loss: 1.1897 - accuracy: 0.6596
Epoch 9/200
10/10 [==============================] - 0s 3ms/step - loss: 1.1585 - accuracy: 0.7021
Epoch 10/200
10/10 [==============================] - 0s 3ms/step - loss: 1.0836 - accuracy: 0.6596
Epoch 11/

## Chatbot Implementation

The chatbot implementation includes a GUI where users input phrases from which the model will predict the class for. This will determine the type of response that the chatbot will produce.

In [11]:
model = load_model('chatbot_model.h5')


def clean_up_sentence(sentence):
    # tokenize the pattern - splitting words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stemming every word - reducing to base form
    sentence_words = [wnl.lemmatize(word.lower()) for word in sentence_words]

    return sentence_words

# return bag of words array: 0 or 1 for words that exist in sentence
def bag_of_words(sentence, words, show_details=True):
    # tokenizing patterns
    sentence_words = clean_up_sentence(sentence)
    # bag of words - vocabulary matrix
    bag = [0]*len(words)
    
    for s in sentence_words:
        for i,word in enumerate(words):
            if word == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % word)

    return(np.array(bag))

def predict_class(sentence):
    # filter below threshold predictions
    p = bag_of_words(sentence, words,show_details=False)

    res = model.predict(np.array([p]))[0]

    ERROR_THRESHOLD = 0.25

    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]

    # sorting strength probability
    results.sort(key=lambda x: x[1], reverse=True)

    return_list = []

    for r in results:

        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})

    return return_list

def getResponse(ints, intents_json):

    tag = ints[0]['intent']

    list_of_intents = intents_json['intents']

    for i in list_of_intents:

        if(i['tag']== tag):

            result = random.choice(i['responses'])
            
            break
        
    return result

#Creating tkinter GUI

import tkinter

from tkinter import *

def send():

    msg = EntryBox.get("1.0",'end-1c').strip()

    EntryBox.delete("0.0",END)

    if msg != '':

        ChatBox.config(state=NORMAL)
        ChatBox.insert(END, "You: " + msg + '\n\n')
        ChatBox.config(foreground="#446665", font=("Verdana", 12 )) 

        ints = predict_class(msg)

        res = getResponse(ints, intents)

        ChatBox.insert(END, "Bot: " + res + '\n\n')           

        ChatBox.config(state=DISABLED)

        ChatBox.yview(END)

root = Tk()
root.title("Chatbot")
root.geometry("400x500")
root.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatBox = Text(root, bd=0, bg="white", height="8", width="50", font="Arial",)
ChatBox.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(root, command=ChatBox.yview, cursor="heart")
ChatBox['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(root, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#f9a602", activebackground="#3c9d9b",fg='#000000',
                    command= send )

#Create the box to enter message
EntryBox = Text(root, bd=0, bg="white",width="29", height="5", font="Arial")

#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatBox.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

root.mainloop()